1.0 PRÁCTICA

1.1 Extracción de datos

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CSV ya arreglado: separado por comas
data = pd.read_csv('data/drugsComTest_raw.csv')

data.head()

data.info()

<class 'pandas.DataFrame'>
RangeIndex: 53766 entries, 0 to 53765
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   53766 non-null  int64  
 1   drugName     53766 non-null  str    
 2   condition    53471 non-null  str    
 3   review       53766 non-null  str    
 4   rating       53766 non-null  float64
 5   date         53766 non-null  str    
 6   usefulCount  53766 non-null  int64  
dtypes: float64(1), int64(2), str(4)
memory usage: 2.9 MB


1.2 Trato de datos NULL

In [17]:
#Tabla rating
# 0) Comprobación rápida antes de limpiar
print("dtype rating (antes):", data["rating"].dtype)
print("nulos rating (antes):", data["rating"].isna().sum())

# 1) Forzar conversión a numérico (lo no-convertible pasa a NaN)
data["rating"] = pd.to_numeric(data["rating"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en rating es: {data['rating'].isna().sum()}")

# 3) Check de rango (ratings deberían estar entre 0 y 10)
print("min rating:", data["rating"].min(), "max rating:", data["rating"].max())

print(f"La mediana de los ratings es: {data['rating'].median()}")
#Como rating no es nuestra variavle explicita y no sesgará el resultado la vamos a modificar
#Utilizaremos la mediana para rellenar los valores nulos 
data["rating"] = data["rating"].fillna(data["rating"].median())



La suma de valores nulos en rating es: 0
La mediana de los ratings es: 8.0
